In [0]:
df_raw = spark.table("trip_data.trip_source.raw")

df_raw.write.format("delta").mode("overwrite").saveAsTable(
    "trip_data.trip_source.bronze"
)


In [0]:
spark.table("trip_data.trip_source.bronze").printSchema()


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [0]:
%sql
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN tpep_pickup_datetime IS NULL THEN 1 END) AS null_pickup,
  SUM(CASE WHEN tpep_dropoff_datetime IS NULL THEN 1 END) AS null_dropoff,
  SUM(CASE WHEN passenger_count IS NULL THEN 1 END) AS null_passenger,
  SUM(CASE WHEN trip_distance IS NULL THEN 1 END) AS null_distance,
  SUM(CASE WHEN fare_amount IS NULL THEN 1 END) AS null_fare
FROM trip_data.trip_source.bronze;


total_rows,null_pickup,null_dropoff,null_passenger,null_distance,null_fare
2964624,null,null,140162,null,null


In [0]:
%sql
SELECT 
  COUNT(*) AS total_rows,
  COUNT(DISTINCT *) AS distinct_rows
FROM trip_data.trip_source.bronze;


total_rows,distinct_rows
2964624,2824462


In [0]:
%sql
SELECT
  SUM(CASE WHEN trip_distance < 0 THEN 1 END) AS negative_distance,
  SUM(CASE WHEN trip_distance = 0 THEN 1 END) AS zero_distance,
  SUM(CASE WHEN fare_amount < 0 THEN 1 END) AS negative_fare
FROM trip_data.trip_source.bronze;


negative_distance,zero_distance,negative_fare
null,60371,37448


In [0]:
%sql
SELECT COUNT(*)
FROM trip_data.trip_source.bronze
WHERE tpep_dropoff_datetime < tpep_pickup_datetime;


COUNT(*)
56


In [0]:
%sql
SELECT 
  MAX(passenger_count),
  MAX(trip_distance),
  MAX(fare_amount)
FROM trip_data.trip_source.bronze;


MAX(passenger_count),MAX(trip_distance),MAX(fare_amount)
9.0,312722.3,5000.0


In [0]:
%sql
SELECT
  MIN(trip_distance) AS min_distance,
  MAX(trip_distance) AS max_distance,
  AVG(trip_distance) AS avg_distance,
  MIN(fare_amount) AS min_fare,
  MAX(fare_amount) AS max_fare
FROM trip_data.trip_source.bronze;


min_distance,max_distance,avg_distance,min_fare,max_fare
0.0,312722.3,3.6521691789582706,-899.0,5000.0


In [0]:
%sql
SELECT DISTINCT payment_type FROM trip_data.trip_source.bronze;


payment_type
2
1
4
3
0


In [0]:
%sql
SELECT *
FROM trip_data.trip_source.bronze
WHERE YEAR(tpep_pickup_datetime) NOT BETWEEN 2020 AND 2025;


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
2,2009-01-01T23:58:40.000Z,2009-01-02T00:01:40.000Z,1.0,0.46,1.0,N,137,264,2,4.4,3.5,0.5,0.0,0.0,1.0,9.4,0.0,0.0
2,2002-12-31T22:59:39.000Z,2002-12-31T23:05:41.000Z,1.0,0.63,1.0,N,170,170,3,-6.5,0.0,-0.5,0.0,0.0,-1.0,-10.5,-2.5,0.0
2,2002-12-31T22:59:39.000Z,2002-12-31T23:05:41.000Z,1.0,0.63,1.0,N,170,170,3,6.5,0.0,0.5,0.0,0.0,1.0,10.5,2.5,0.0
2,2009-01-01T23:30:39.000Z,2009-01-02T00:01:39.000Z,1.0,10.99,1.0,N,237,264,2,45.0,3.5,0.5,0.0,0.0,1.0,50.0,0.0,0.0
2,2009-01-01T00:24:09.000Z,2009-01-01T01:13:00.000Z,2.0,10.88,1.0,N,138,264,2,50.6,9.25,0.5,0.0,6.94,1.0,68.29,0.0,0.0
